# My portfolios

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import robin_stocks as r
import etfs as et

In [ ]:
from etfs.portfolio.io import import_portfolio, import_portfolio_robinhood, import_portfolio_vanguard, \
                              parse_portfolio, parse_portfolio_vanguard
from etfs.stats.basics import resample_df, runrate_column, shift_column, \
                              standard_deviation_column, difference, rsq, beta, alpha
from etfs.utils.helpers import todays_date
from etfs.visualization.trends import plot_composition
from etfs.visualization.analysis import plot_cross_correlation_matrix

## RobinHood

In [ ]:
# access password file
f = open("../../../.etfs.txt","r")
lines = f.readlines()
username = lines[0].strip().split('\n', 0)
password = lines[1].strip().split('\n', 0)
f.close()

In [ ]:
access_token = r.login(username, password)

In [ ]:
rh = import_portfolio_robinhood(access_token=access_token,  
                                name="Robinhood", 
                                free_stock=True)

In [ ]:
rh.overview()

In [ ]:
rh.overview_archive()

In [ ]:
# parse alternative (SP500) portfolio first
rh_alt = import_portfolio(path="../data/portfolio_rh_alt.csv", name="RobinHood")

In [ ]:
rh_alt.overview()

In [ ]:
rh.payments.sum()

In [ ]:
rh_alt.payments.sum()

In [ ]:
rh.positions()

In [ ]:
rh.get_timeseries()

In [ ]:
rh.data.plot(y=[column for column in rh.data.columns if column not in ['Total', 'Total_sh1', 'Total_sh1_ret', 'Total_sh1_sqerr']], 
               figsize=(12,12)
            )
#plt.ylim((0,2000))
#plt.yscale('symlog')

In [ ]:
plot_composition(portfolio=rh)

In [ ]:
rh.data_growth.plot(y="Growth")

In [ ]:
rh_alt.get_timeseries()

In [ ]:
plt.plot(rh.data_growth.Growth[:], label='MyPortfolio')
plt.plot(rh_alt.data_growth.Growth[:], label='SP500')
plt.legend()

In [ ]:
plt.plot(rh.data_growth.Growth-rh_alt.data_growth.Growth)
plt.hlines(0, xmin=min(rh.data_growth.index), xmax=max(rh.data_growth.index), linestyle='-', linewidth=0.5)

In [ ]:
rh_alt.get_performance()

In [ ]:
rh.get_performance()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(rh.data, column='Total', window=1, shift=1)

In [ ]:
rh.get_benchmark()

In [ ]:
rh.get_returns()

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=rh.data) # to-do: turn this into portfolio object

In [ ]:
df_volatility = standard_deviation_column(total.data, column='Total', window=1, shift=1)

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
total.benchmark.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=rh)

## Put in stop loss orders for Robinhood stocks

In [ ]:
#from etfs.trader.orders import put_in_stop_loss_orders_all

put_in_stop_loss_orders_all(access_token=access_token, 
                            portfolio=rh, 
                            column="Close", 
                            alpha=.5, 
                            sigmas=1)

## Vanguard

In [ ]:
vg = import_portfolio_vanguard(path="../data/portfolio_vanguard*.csv", name="Vanguard")

In [ ]:
vg.overview()     

In [ ]:
vg.overview_archive()

In [ ]:
vg.positions()

In [ ]:
vg.get_timeseries()

In [ ]:
ax = vg.data.plot(y=vg.data.columns,
                  figsize=(8,6)
                 )

In [ ]:
plot_composition(portfolio=vg)

In [ ]:
vg.data_growth.plot(y="Growth")

In [ ]:
vg.get_performance()

In [ ]:
vg.get_benchmark()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(vg.data, column='Total', window=1, shift=1)

In [ ]:
df_volatility = standard_deviation_column(vg.benchmark.data, column='Close', window=1, shift=1)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=vg.data) # to-do: turn this into portfolio object

In [ ]:
total.get_benchmark()

In [ ]:
total.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=vg)

## Add up portfolios

In [ ]:
tot = et.Portfolio(name="tot")

In [ ]:
tot = parse_portfolio([rh.transactions, vg.transactions], tot)

In [ ]:
tot.get_timeseries()

In [ ]:
tot.data_growth.plot(y=["Growth"])

In [ ]:
tot.get_performance()

In [ ]:
tot.overview()

In [ ]:
tot.positions()

In [ ]:
tot.overview_df['Growth'] = tot.overview_df['Return']/tot.overview_df['CurrentValue']*100

In [ ]:
ax = tot.overview_df.sort_values(by='Growth', ascending=True).plot(x='Description', 
                                                                   y='Growth', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Growth',
                                                                   figsize=(10,8)
                                                                  )

In [ ]:
ax = tot.positions_df.reset_index().sort_values(by='PercentGrowth', ascending=True).plot(x='Ticker', 
                                                                                         y='PercentGrowth', 
                                                                                         kind='barh', 
                                                                                         legend=False, 
                                                                                         title='PercentGrowth',
                                                                                         figsize=(10,10)
                                                                                        )

In [ ]:
tot.overview_df['PriceChange'] = 100-tot.overview_df['AvgPriceFiFo']/tot.overview_df['LastPrice']*100

In [ ]:
ax = tot.overview_df.sort_values(by='PriceChange', ascending=True).plot(x='Description', 
                                                                        y='PriceChange', 
                                                                        kind='barh', 
                                                                        legend=False, 
                                                                        title='Price Change',
                                                                        figsize=(10,8)
                                                                       )

In [ ]:
tot.overview_archive()

In [ ]:
ax = tot.overview_archive_df.sort_values(by='Return', ascending=True).plot(x='Description', 
                                                                   y='Return', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Return',
                                                                   figsize=(10,8)
                                                                  )

In [ ]:
ax = pd.concat([tot.overview_df, tot.overview_archive_df], axis=0, ignore_index=True, sort=False).sort_values(by='Return', ascending=True).plot(x='Description', 
                                                                   y='Return', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Return',
                                                                   figsize=(10,10)
                                                                  )

In [ ]:
plot_composition(portfolio=tot)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=tot.data) # to-do: turn this into portfolio object

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
rsq(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=tot)

## Performance analysis of individual securities

In [ ]:
from etfs.visualization.analysis import plot_security_performance

In [ ]:
plot_security_performance(portfolio=tot, ticker='OKTA')

In [ ]:
for ticker in tot.tickers:
    plot_security_performance(portfolio=tot, ticker=ticker)